# automated FEAT
* **add these fields to your database for each run: MAKE SURE THE NAME ARE EXACTLY AS BELOW**
 * field_map_phase
 * field_map_mag
 * anatomical
 * dwell
 * TE
* **assumptions**
 * there is only one nifti file in the run_data_file directory in your spreadsheet
 * template. fsf is in the same directory as this notebook
 * feat_automation_utils.py is in the same directory as this notebook
 

# load spreadsheet
<ol>
<li> set db equal to path to your csv file(same as from other pipeline/etc.) </li>
<li> set subject equal to subject initials</li>
<li> run the cell</li>
</ol>

In [1]:
from feat_automation_utils import *

## put your spreadsheet here
db = '/home/nick/Desktop/CO_dataset/db.csv'

# put subject initials here
subject = 'CO'

df = load_db(db,subject)
df

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days


,runID,date,subject,expID,working_vol,brain_mask,sessionID,runType,design matrix,frame_file,run_data_file,nvols,tesla,location,run_path,TR,matrix_x,matrix_y,n_slices,vox_x,vox_y,vox_z,run_code_path,run_code_file,design_matrix_path,frame_file_path,pic_path,siemensRef,padVol,field_map_mag,field_map_phase,anatomical,dwell,TE
5,142,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,trn,onoffTrn.mat,trn001.txt,1_004_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,210,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_gre_field_mapping_1x1...,/home/nick/Desktop/CO_s1000_T1.nii,0.3,22.8
6,143,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,trn,onoffTrn.mat,trn002.txt,1_005_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,210,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_gre_field_mapping_1x1...,/home/nick/Desktop/CO_s1000_T1.nii,0.3,22.8
7,144,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,trn,onoffTrn.mat,trn003.txt,1_006_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,210,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_gre_field_mapping_1x1...,/home/nick/Desktop/CO_s1000_T1.nii,0.3,22.8
8,145,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,val,onoffVal.mat,val001.txt,1_007_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,210,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_gre_field_mapping_1x1...,/home/nick/Desktop/CO_s1000_T1.nii,0.3,22.8
9,146,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,imgVal,onoffImg.mat,imgVal.txt,1_008_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,232,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_gre_field_mapping_1x1...,/home/nick/Desktop/CO_s1000_T1.nii,0.3,22.8
10,147,10/13/2011,CO,lotusHill.7T.em9K.1.5iso,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,/home/nick/Desktop/CO_dataset/lotusHill.7T.em9...,11,img,onoffImg.mat,img.txt,1_009_eja_ep2d_bold_1_5mm_2000TR_FH_20111013,232,7T,CMRR,/home/nick/ichi/mnt/vison_conv/naselaris_ca,2,108,108,39,1.5,1.5,1.5,/Data/7T.cmrr/Presentation/buffermovie02082010,Oct.13.2011.7T.CA.py,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/Oct2011,/Data/7T.cmrr/Presentation/pictures,1,0,/home/nick/Desktop/1_015_gre_field_mapping_1x1...,/home/nick/Desktop/1_016_

#Choose an output directory

In [5]:
inputs = dict()
inputs['out_dir'] = '/home/nick/Desktop/feat_out'

#Options
* pretty much same as the gui
* self explanatory I think, True for yes, False for no.
* make sure True and False are capitalized.

In [6]:

inputs['Motion Correction'] = True

inputs['B0 Unwarping'] = True

inputs['Brain Extraction'] = True

inputs['Register to Structural'] = True

inputs['Spatial Smooth'] = 5 # Spatial smoothing FWHM (mm)

inputs['BBR DOF'] = 3 # Degrees of Freedom for registration to initial structural

inputs['Unwarp Direction'] = 'y-'

# here the feat run files are built, don't need to do anything

In [7]:
runs = build_runs(df,inputs)
fsfs = [build_fsf(run[0],run[1],get_template()) for run in runs]
fsf_files = save_fsf(inputs['out_dir'],fsfs)


# this is where feat is actually run, just run the cell and we're off
this should act equivalently to pressing go in the gui, but it will run for all the runs.

In [ ]:
run_feat_all(fsf_files)